In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from copy import deepcopy as copy

In [ ]:
# initial cost for a tesla, currently $39k in real life, Elon promised $25k by 2022
cost_for_tesla = 39_000
# how many miles each car can operate for before being de-commisioned
tesla_longevity_miles = 1_000_000
# how many miles the car is operational each month 7,500 in Tesla slides
tesla_miles_per_month = 7_500
number_of_months = 20 * 12
# 0.65 in Tesla slides
profit_per_mile = 0.65
# 0.5 in Tesla slides
occupied_miles = 0.5
# change in profit each year
profit_decay_coefficient = 0.95
# change in tesla price each year
tesla_price_increase_coefficient = 0.03
# what order of magnitude steps up operational costs (if set to 10 will do 10, 100, 1000, etc...)
magnitude_adjustment_original = 10
current_magnitude_adjustment = magnitude_adjustment_original
# change in each tesla cost when next order of magnitude is hit
magnitude_step_change = 0.05

In [ ]:

number_of_teslas = 1
cash = 0 - (number_of_teslas*cost_for_tesla)
remaining_miles = []
cash


In [ ]:
for x in range(0, number_of_teslas):
    remaining_miles.append(tesla_longevity_miles)
remaining_miles

In [ ]:
# currently represents total revenue, comment out this line and uncomment the line below to make it only money made afer debt is paid
total_money_earned = 0
# total_money_earned = cash
df= pd.DataFrame(columns=["Month", "Number of Teslas Owned", "Operational Miles Remaining", "Total Money Earned", "Cash"])
for x in range(0, number_of_months):
    print(x)
    # annual adjustment
    if x/12 == x//12:
        profit_per_mile = profit_per_mile * profit_decay_coefficient
        cost_for_tesla = cost_for_tesla * (1 + tesla_price_increase_coefficient)
    money_earned_month = 0
    new_remaining = []
    for y in remaining_miles:
        if y>= tesla_miles_per_month:
            money_earned_month = money_earned_month + (occupied_miles * profit_per_mile*tesla_miles_per_month)
            remaining_miles = y- tesla_miles_per_month
            if remaining_miles>0:
                new_remaining.append(remaining_miles)
        else:
            money_earned_month = money_earned_month + (occupied_miles * profit_per_mile * y)
    total_money_earned = total_money_earned + money_earned_month
    cash = cash + money_earned_month
    if cash>=cost_for_tesla:
        number_of_teslas_to_buy = int(cash//cost_for_tesla)
        # print(f"Number of Teslas to buy: {number_of_teslas_to_buy}")
        cash = cash - number_of_teslas_to_buy*cost_for_tesla
        for z in range(0, number_of_teslas_to_buy):
            new_remaining.append(tesla_longevity_miles)
    remaining_miles = copy(new_remaining)
    if len(remaining_miles) >= current_magnitude_adjustment:
        cost_for_tesla = cost_for_tesla * (1 + magnitude_step_change)
        current_magnitude_adjustment = current_magnitude_adjustment * magnitude_adjustment_original
    dictionary = {
        "Month": x+1,
        "Number of Teslas Owned": len(remaining_miles),
        "Operational Miles Remaining": sum(remaining_miles),
        "Total Money Earned": total_money_earned,
        "Cash": cash
    }
    df = df.append(dictionary, ignore_index=True)
df

In [ ]:
df.to_csv('robotaxi_analysis_39k_decay.csv', index=False)